# Split Half

In [1]:
import pandas
import numpy
import multiprocess

In [2]:
df = pandas.read_csv('./DATA/df1.csv')

In [3]:
# 创建一个空list用于存放分半的结果
shl = [[] for _ in range(10)]

# 单线程分半
for i in range(10):

    # 按照指定的四个变量进行分组
    grouped = df.groupby(['Subject', 'Matching', 'Identity', 'Session'])

    # 设置随机种子
    numpy.random.seed(i)

    # 定义拆分函数
    def split(group):
        indices = numpy.random.permutation(len(group))  # 随机打乱索引
        if len(indices) % 2 != 0:  # 检查索引长度是否为奇数
            indices = indices[:-1]  # 如果是，就删除最后一个索引
        else:
            indices = indices # 偶数就保留原索引
        half = len(indices) // 2 # 得到索引的一半位置是多少
        return group.iloc[indices[:half]].reset_index(drop=True), group.iloc[indices[half:]].reset_index(drop=True)

    # 创建两个空DataFrame来存储拆分后的结果
    result1 = pandas.DataFrame()
    result2 = pandas.DataFrame()

    # 对每个组应用拆分函数，并将结果存储到对应的DataFrame中
    for (_, _, _, _), (group1, group2) in grouped.apply(split).items():
        result1 = pandas.concat([result1, group1], ignore_index=True)
        result2 = pandas.concat([result2, group2], ignore_index=True)

    # 解除分组
    result1 = result1.reset_index(drop=True)
    result2 = result2.reset_index(drop=True)
    
    shl[i].append(result1)  # 添加 result1 到 shl[i] 的列表中
    shl[i].append(result2)  # 添加 result2 到 shl[i] 的列表中

del group1, group2, grouped, i, result1, result2

In [4]:
# 多线程(函数)
def process_iteration(i):
    
    import pandas
    import numpy
    df = pandas.read_csv('./DATA/df1.csv')
 
    # 按照指定的四个变量进行分组
    grouped = df.groupby(['Subject', 'Matching', 'Identity', 'Session'])

    # 创建一个空list用于存放分半的结果
    shl = [[] for _ in range(i)]

    # 设置随机种子
    numpy.random.seed(i)

    # 定义拆分函数
    def split(group):
        indices = numpy.random.permutation(len(group))  # 随机打乱索引
        if len(indices) % 2 != 0:  # 检查索引长度是否为奇数
            indices = indices[:-1]  # 如果是，就删除最后一个索引
        else:
            indices = indices # 偶数就保留原索引
        half = len(indices) // 2 # 得到索引的一半位置是多少
        return group.iloc[indices[:half]].reset_index(drop=True), group.iloc[indices[half:]].reset_index(drop=True) # 以乱序索引的中点二分

    # 创建两个空DataFrame来存储拆分后的结果
    result1 = pandas.DataFrame()
    result2 = pandas.DataFrame()

    # 对每个组应用拆分函数，并将结果存储到对应的DataFrame中
    for (_, _, _, _), (group1, group2) in grouped.apply(split).items():
        result1 = pandas.concat([result1, group1], ignore_index=True)
        result2 = pandas.concat([result2, group2], ignore_index=True)

    # 解除分组
    result1 = result1.reset_index(drop=True)
    result2 = result2.reset_index(drop=True)

    return result1, result2

In [5]:
# 多线程(执行)
if __name__ == '__main__':
    num_iterations = 10
    pool = multiprocess.Pool(processes=16)

    results = pool.map(process_iteration, range(num_iterations))

    shl = [[] for _ in range(num_iterations)]
    for i, (result1, result2) in enumerate(results):
        shl[i].append(result1)
        shl[i].append(result2)

del i, num_iterations, pool, result1, result2, results,

# PyDDM

In [18]:
import pyddm